# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [247]:
# Reading in csv
accidents_df = pd.read_csv("AviationData.csv", encoding="1252")

# Checking Data
accidents_df.head()

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\3365125358.py:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents_df = pd.read_csv("AviationData.csv", encoding="1252")


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [248]:
# Getting DF info
accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [249]:
# converting Event.Dates to Datetime dtype
accidents_df["Event.Date"] = pd.to_datetime(accidents_df["Event.Date"])

# Remove data from greater than 40 years prior
date_cleaned_df = accidents_df.loc[accidents_df["Event.Date"] > "1983-01-01"] 

In [250]:
#Checking for duplicate entries
date_cleaned_df["Event.Id"].value_counts()

Event.Id
20001212X19172    3
20001214X45071    3
20001213X28363    2
20001213X32577    2
20001213X29734    2
                 ..
20001211X12036    1
20001211X11969    1
20001211X12014    1
20001211X12027    1
20221230106513    1
Name: count, Length: 84391, dtype: int64

In [251]:
# Removing repeated entries
no_duplicated_events = date_cleaned_df.drop_duplicates(subset="Event.Id")

In [252]:
#Recheck
no_duplicated_events["Event.Id"].value_counts()

Event.Id
20001214X42064    1
20060607X00697    1
20060618X00761    1
20060525X00622    1
20060618X00764    1
                 ..
20001211X11796    1
20001211X11767    1
20001211X11800    1
20001211X11864    1
20221230106513    1
Name: count, Length: 84391, dtype: int64

In [253]:
# Removing records of Amateur Built aircraft
professional_builds = no_duplicated_events[no_duplicated_events["Amateur.Built"] != "Yes"]
professional_builds = professional_builds.drop(["Amateur.Built"], axis=1)

In [254]:
# Removing craft listed as non-airplane
professional_builds.fillna({"Aircraft.Category":"Airplane"}, inplace=True)
professional_airplanes1 = professional_builds.loc[professional_builds["Aircraft.Category"]=="Airplane"]

In [255]:
#Removing "Balloon Works" aircraft as non-airplane
professional_airplanes = professional_airplanes1.loc[professional_airplanes1["Make"]!="Balloon Works"]

## Data Cleaning

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [256]:
# Calculating and printing mean and median for injury columns
fatal_mean = professional_airplanes["Total.Fatal.Injuries"].mean()
serious_mean = professional_airplanes["Total.Serious.Injuries"].mean()
minor_mean = professional_airplanes["Total.Minor.Injuries"].mean()
uninjured_mean = professional_airplanes["Total.Uninjured"].mean()

fatal_median = professional_airplanes["Total.Fatal.Injuries"].median()
serious_median = professional_airplanes["Total.Serious.Injuries"].median()
minor_median = professional_airplanes["Total.Minor.Injuries"].median()
uninjured_median = professional_airplanes["Total.Uninjured"].median()

print("Injury Means:")
print(fatal_mean, serious_mean, minor_mean, uninjured_mean)
print("Injury Medians:")
print(fatal_median, serious_median, minor_median, uninjured_median)

Injury Means:
0.6736562834630568 0.2795845504706264 0.36303024446806453 5.903752281191116
Injury Medians:
0.0 0.0 0.0 1.0


In [257]:
# Replacing nulls with 0 for injury columns with medians of 0 and means less than 1
professional_airplanes["Total.Fatal.Injuries"] = professional_airplanes["Total.Fatal.Injuries"].fillna(0)
professional_airplanes["Total.Serious.Injuries"] = professional_airplanes["Total.Fatal.Injuries"].fillna(0)
professional_airplanes["Total.Minor.Injuries"] = professional_airplanes["Total.Fatal.Injuries"].fillna(0)

# Replacing uninjured with the median value of 1
professional_airplanes["Total.Uninjured"] = professional_airplanes["Total.Uninjured"].fillna(1)

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2790052171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  professional_airplanes["Total.Fatal.Injuries"] = professional_airplanes["Total.Fatal.Injuries"].fillna(0)
C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2790052171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  professional_airplanes["Total.Serious.Injuries"] = professional_airplanes["Total.Fatal.Injuries"].fillna(0)
C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2790052171.py:4: Setti

In [258]:
# Renaming dataframe for ease of coding
airplane_df = professional_airplanes

In [259]:
# Creating total passenger column
def total_passenger(row):
    return row["Total.Fatal.Injuries"]+row["Total.Serious.Injuries"]+row["Total.Minor.Injuries"]+row["Total.Uninjured"]

airplane_df["Total.Passengers"] = airplane_df.apply(total_passenger, axis=1)

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\3023909116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_df["Total.Passengers"] = airplane_df.apply(total_passenger, axis=1)


In [260]:
# Filtering out any results with Total Passengers equal to zero
airplane_filtered = airplane_df[airplane_df["Total.Passengers"] != 0]

In [261]:
# Creating ratio columns for fatal and serious unjuries
def fatal_ratio(row):
    return row["Total.Fatal.Injuries"]/row["Total.Passengers"]

def serious_ratio(row):
    return row["Total.Serious.Injuries"]/row["Total.Passengers"]

def fatal_and_serious_ratio(row):
    return row["Fatality Ratio"]+row["Serious Injury Ratio"]
    
airplane_filtered["Fatality Ratio"] = airplane_filtered.apply(fatal_ratio, axis=1)
airplane_filtered["Serious Injury Ratio"] = airplane_filtered.apply(serious_ratio, axis=1)
airplane_filtered["Fatality/Serious Injury Ratio"] = airplane_filtered.apply(fatal_and_serious_ratio, axis=1)

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2730106192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_filtered["Fatality Ratio"] = airplane_filtered.apply(fatal_ratio, axis=1)
C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2730106192.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_filtered["Serious Injury Ratio"] = airplane_filtered.apply(serious_ratio, axis=1)
C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\2730106192.py:13: SettingWithCopyWarning: 
A value is trying

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [262]:
# Checking Aircraft Damage collumn
airplane_filtered["Aircraft.damage"].value_counts()

Aircraft.damage
Substantial    43552
Destroyed      12235
Minor           2093
Unknown           61
Name: count, dtype: int64

In [263]:
# Boolean column created for True if Aircraft listed as destroyed
airplane_filtered["Aircraft Destroyed?"] = airplane_filtered["Aircraft.damage"]=="Destroyed"

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\4074366189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_filtered["Aircraft Destroyed?"] = airplane_filtered["Aircraft.damage"]=="Destroyed"


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [264]:
# Dropping nulls from make
airplane_filtered = airplane_filtered.dropna(subset=["Make"])

In [265]:
# Checking Make values
airplane_filtered["Make"].value_counts().head(50)

Make
Cessna                            17471
Piper                              9407
CESSNA                             4050
Beech                              3526
PIPER                              2308
Bell                               1382
Boeing                             1361
BEECH                               854
Mooney                              852
Grumman                             790
BOEING                              665
Bellanca                            658
Robinson                            576
Hughes                              542
Air Tractor                         466
Mcdonnell Douglas                   464
Schweizer                           423
Maule                               395
Aeronca                             348
Champion                            334
De Havilland                        325
Aero Commander                      284
Stinson                             266
Rockwell                            253
North American                     

In [266]:
# Stripping and capitalizing Make Names
airplane_filtered["Make"] = airplane_filtered["Make"].str.strip()
airplane_filtered["Make"] = airplane_filtered["Make"].str.capitalize()

# Standardizing Airbus, Mcdonnel douglas, Bombardier, Aviat, and de Havilland
airplane_filtered["Make"] = airplane_filtered["Make"].replace("Airbus industrie", "Airbus")
airplane_filtered["Make"] = airplane_filtered["Make"].replace("Dehavilland", "De havilland")
airplane_filtered["Make"] = airplane_filtered["Make"].replace("Douglas", "Mcdonnell douglas")
airplane_filtered["Make"] = airplane_filtered["Make"].replace("Bombardier inc", "Bombardier")
airplane_filtered["Make"] = airplane_filtered["Make"].replace("Aviat aircraft inc", "Aviat")

In [267]:
# Recheck
airplane_filtered["Make"].value_counts().head(50)

Make
Cessna                            21521
Piper                             11715
Beech                              4380
Boeing                             2026
Bell                               1392
Mooney                             1043
Grumman                             845
Bellanca                            784
Mcdonnell douglas                   749
Robinson                            591
Hughes                              543
Air tractor                         535
Maule                               520
Aeronca                             462
De havilland                        449
Schweizer                           428
Champion                            408
Stinson                             336
Aero commander                      333
North american                      311
Luscombe                            310
Airbus                              294
Rockwell                            276
Taylorcraft                         267
Aerospatiale                       

In [268]:
# Removing Makes with fewer than 50 mentions
value_counts = airplane_filtered["Make"].value_counts()
restricted_value_counts = value_counts>=50
restricted_value_counts = restricted_value_counts[restricted_value_counts == False]
drop_list = restricted_value_counts.index

airplane_makes_cut = airplane_filtered[~airplane_filtered["Make"].isin(drop_list)]

In [269]:
# Check
airplane_makes_cut["Make"].value_counts()

Make
Cessna        21521
Piper         11715
Beech          4380
Boeing         2026
Bell           1392
              ...  
Pitts            58
Fokker           55
Ercoupe          54
Schleicher       50
Mbb              50
Name: count, Length: 70, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [270]:
# Drop Model Nulls

airplane_model_cut = airplane_makes_cut.dropna(subset=["Model"])

In [271]:
# Check
airplane_model_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55378 entries, 3608 to 88888
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Event.Id                       55378 non-null  object        
 1   Investigation.Type             55378 non-null  object        
 2   Accident.Number                55378 non-null  object        
 3   Event.Date                     55378 non-null  datetime64[ns]
 4   Location                       55338 non-null  object        
 5   Country                        55228 non-null  object        
 6   Latitude                       20444 non-null  object        
 7   Longitude                      20440 non-null  object        
 8   Airport.Code                   32433 non-null  object        
 9   Airport.Name                   33934 non-null  object        
 10  Injury.Severity                55378 non-null  object        
 11  Aircraft.damage  

In [272]:
# Stripping Model strings and checking counts
airplane_model_cut["Model"] = airplane_model_cut["Model"].str.strip()
airplane_model_cut["Model"].value_counts()

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\3216716133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_model_cut["Model"] = airplane_model_cut["Model"].str.strip()


Model
152          1871
172          1368
172N          918
PA-28-140     644
172M          620
             ... 
305B            1
747-368         1
BF12-65         1
B206BII         1
P63             1
Name: count, Length: 5063, dtype: int64

In [273]:
# Creating combined "Make - Model" column

airplane_model_cut["Make - Model"] = airplane_model_cut["Make"] + " - " + airplane_model_cut["Model"]

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\1889989705.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_model_cut["Make - Model"] = airplane_model_cut["Make"] + " - " + airplane_model_cut["Model"]


In [274]:
airplane_model_cut["Make - Model"].value_counts()

Make - Model
Cessna - 152                        1871
Cessna - 172                        1368
Cessna - 172N                        918
Piper - PA-28-140                    644
Cessna - 172M                        620
                                    ... 
Cessna - 206B                          1
Piper - PA-25-225                      1
Burkhart grob - G-103 TWIN ASTIR       1
Aerospatiale - SA-365-N2               1
Bell - P63                             1
Name: count, Length: 5422, dtype: int64

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [275]:
# Checking engine types
airplane_model_cut["Engine.Type"].value_counts()

Engine.Type
Reciprocating      44330
Turbo Prop          2435
Turbo Fan           1941
Turbo Shaft         1642
Unknown             1031
Turbo Jet            505
Geared Turbofan        1
Name: count, dtype: int64

In [276]:
# Checking Weather conditions
airplane_model_cut["Weather.Condition"].value_counts()

Weather.Condition
VMC    47767
IMC     4545
UNK      613
Unk      166
Name: count, dtype: int64

In [277]:
# Taking Weather conditions to all caps
airplane_model_cut["Weather.Condition"] = airplane_model_cut["Weather.Condition"].str.upper()

C:\Users\jesch\AppData\Local\Temp\ipykernel_27868\3452080468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airplane_model_cut["Weather.Condition"] = airplane_model_cut["Weather.Condition"].str.upper()


In [278]:
# Weather recheck
airplane_model_cut["Weather.Condition"].value_counts()

Weather.Condition
VMC    47767
IMC     4545
UNK      779
Name: count, dtype: int64

In [279]:
# number of engines check
airplane_model_cut["Number.of.Engines"].value_counts()

Number.of.Engines
1.0    42725
2.0     8285
3.0      389
0.0      366
4.0      364
Name: count, dtype: int64

In [280]:
# Removing rows with 0 engines listed
airplane_model_cut = airplane_model_cut[airplane_model_cut["Number.of.Engines"] != 0]

In [281]:
# recheck
airplane_model_cut["Number.of.Engines"].value_counts()

Number.of.Engines
1.0    42725
2.0     8285
3.0      389
4.0      364
Name: count, dtype: int64

In [282]:
# Checking purpose of flight:
airplane_model_cut["Purpose.of.flight"].value_counts()

Purpose.of.flight
Personal                     29450
Instructional                 7499
Unknown                       4545
Aerial Application            2939
Business                      2768
Positioning                   1090
Other Work Use                 698
Ferry                          470
Public Aircraft                464
Aerial Observation             435
Executive/corporate            363
Skydiving                      157
Flight Test                    117
Banner Tow                      70
Public Aircraft - Federal       44
Public Aircraft - State         26
Glider Tow                      25
Firefighting                    21
Air Race/show                   14
Public Aircraft - Local         14
Air Race show                   13
External Load                   12
Air Drop                         5
PUBS                             2
ASHO                             2
Name: count, dtype: int64

In [283]:
# Checking phase of flight:
airplane_model_cut["Broad.phase.of.flight"].value_counts()

Broad.phase.of.flight
Landing        11632
Takeoff         7771
Cruise          6758
Maneuvering     4741
Approach        3881
Taxi            1457
Climb           1305
Descent         1253
Go-around        890
Standing         686
Unknown          360
Other             70
Name: count, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [284]:
airplane_model_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55012 entries, 3608 to 88888
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Event.Id                       55012 non-null  object        
 1   Investigation.Type             55012 non-null  object        
 2   Accident.Number                55012 non-null  object        
 3   Event.Date                     55012 non-null  datetime64[ns]
 4   Location                       54974 non-null  object        
 5   Country                        54864 non-null  object        
 6   Latitude                       20397 non-null  object        
 7   Longitude                      20393 non-null  object        
 8   Airport.Code                   32316 non-null  object        
 9   Airport.Name                   33811 non-null  object        
 10  Injury.Severity                55012 non-null  object        
 11  Aircraft.damage  

In [285]:
# Longitude, Latitude, Airport.Code, Airport.Name, FAR.Decription, Schedule, Air.Carrier removed
# Reason: Greater than 20,000 null and not inherently relevant

column_cut_df = airplane_model_cut.drop(["Longitude", "Latitude", "Airport.Code", "Airport.Name", "FAR.Description", "Schedule", "Air.carrier"], axis = 1)

In [286]:
column_cut_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55012 entries, 3608 to 88888
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Event.Id                       55012 non-null  object        
 1   Investigation.Type             55012 non-null  object        
 2   Accident.Number                55012 non-null  object        
 3   Event.Date                     55012 non-null  datetime64[ns]
 4   Location                       54974 non-null  object        
 5   Country                        54864 non-null  object        
 6   Injury.Severity                55012 non-null  object        
 7   Aircraft.damage                53106 non-null  object        
 8   Aircraft.Category              55012 non-null  object        
 9   Registration.Number            53985 non-null  object        
 10  Make                           55012 non-null  object        
 11  Model            

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [287]:
column_cut_df.to_csv("Cleaned_Accident_Data.csv", index=False)